# Featuretools : Automated Feature Engineering

Thera are six different datasets in addition to train and test data. To improve performance, we need to methodically integrate those dataset into the train and the test data. Here I am going to use Featuretools library to incorportate those datasets. 

In [1]:
# Importing necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce
import re as re

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler



from sklearn.multiclass import  OneVsRestClassifier

from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
from sklearn.pipeline import FeatureUnion

from sklearn.pipeline import Pipeline
 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb
import gc

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

In [2]:
# A function to reduce memory. 
# Acknowledgement to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

 ##### I am using only a small subset of data so that I can perform all the operation fast. Before putting it into our model, I will use the whole dataset.  

In [11]:
train = reduce_mem(pd.read_csv(
    "DataSet/application_train.csv",
    encoding = 'unicode_escape').sort_values(
    "SK_ID_CURR").reset_index(
    drop = True).loc[:1000,:])
train.head()

Memory usage of dataframe is 0.93 MB
Memory usage after optimization is: 0.30 MB
Decreased by 67.7%


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
test = reduce_mem(pd.read_csv(
    "DataSet/application_test.csv",
    encoding = 'unicode_escape').sort_values(
    "SK_ID_CURR").reset_index(
    drop = True).loc[:1000,:])
test.head()


Memory usage of dataframe is 0.92 MB
Memory usage after optimization is: 0.30 MB
Decreased by 67.6%


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
bureau = reduce_mem(pd.read_csv(
    "DataSet/bureau.csv",
    encoding = 'unicode_escape').sort_values(
    ["SK_ID_CURR", "SK_ID_BUREAU"]).reset_index(
    drop = True).loc[:1000,:])
bureau.head()

Memory usage of dataframe is 0.13 MB
Memory usage after optimization is: 0.06 MB
Decreased by 52.2%


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
1,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0
2,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0
3,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
4,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5


In [15]:
bureau_balance = reduce_mem(pd.read_csv(
    "DataSet/bureau_balance.csv",
    encoding = 'unicode_escape').sort_values(
    "SK_ID_BUREAU").reset_index(
    drop = True).loc[:1000,:])
bureau_balance.head()


Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.01 MB
Decreased by 45.7%


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5001709,-7,C
1,5001709,-3,C
2,5001709,-4,C
3,5001709,-5,C
4,5001709,-6,C


In [17]:
POS_CASH_balance= reduce_mem(pd.read_csv(
    "DataSet/POS_CASH_balance.csv",
    encoding = 'unicode_escape').sort_values(
    ["SK_ID_CURR", "SK_ID_PREV"]).reset_index(
    drop = True).loc[:1000,:])

POS_CASH_balance.head()

Memory usage of dataframe is 0.06 MB
Memory usage after optimization is: 0.02 MB
Decreased by 62.4%


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1369693,100001,-53,4.0,0.0,Completed,0,0
1,1369693,100001,-54,4.0,1.0,Active,0,0
2,1369693,100001,-57,4.0,4.0,Active,0,0
3,1369693,100001,-55,4.0,2.0,Active,0,0
4,1369693,100001,-56,4.0,3.0,Active,0,0


In [23]:
credit_card_balance= reduce_mem(pd.read_csv(
    "DataSet/credit_card_balance.csv",
    encoding = 'unicode_escape').sort_values(
    ["SK_ID_CURR","SK_ID_PREV"]).reset_index(
    drop = True).loc[:1000,:])
credit_card_balance.head()

Memory usage of dataframe is 0.18 MB
Memory usage after optimization is: 0.07 MB
Decreased by 59.2%


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1489396,100006,-2,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1,1489396,100006,-1,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2,1489396,100006,-5,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3,1489396,100006,-3,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
4,1489396,100006,-4,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0


In [45]:
previous_application= reduce_mem(pd.read_csv(
    "DataSet/previous_application.csv",
    encoding = 'unicode_escape').sort_values(
    ["SK_ID_CURR", "SK_ID_PREV"]).reset_index(
    drop = True).loc[:1000,:])
previous_application.head()

Memory usage of dataframe is 0.28 MB
Memory usage after optimization is: 0.18 MB
Decreased by 35.1%


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,1369693,100001,Consumer loans,3951.000000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
1,1038818,100002,Consumer loans,9251.775391,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
2,1810518,100003,Cash loans,98356.992188,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
3,2396755,100003,Consumer loans,6737.310059,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,2636178,100003,Consumer loans,64567.664062,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0


In [24]:
installments_payments= reduce_mem(pd.read_csv(
    "DataSet/installments_payments.csv",
    encoding = 'unicode_escape').sort_values(
    ["SK_ID_CURR", "SK_ID_PREV"]).reset_index(
    drop = True).loc[:1000,:])
installments_payments.head()

Memory usage of dataframe is 0.06 MB
Memory usage after optimization is: 0.02 MB
Decreased by 64.0%


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1369693,100001,1.0,1,-1709.0,-1715.0,3951.000000,3951.000000
1,1369693,100001,1.0,2,-1679.0,-1715.0,3951.000000,3951.000000
2,1369693,100001,2.0,4,-1619.0,-1628.0,17397.900391,17397.900391
3,1369693,100001,1.0,3,-1649.0,-1660.0,3951.000000,3951.000000
4,1851984,100001,1.0,2,-2916.0,-2916.0,3982.050049,3982.050049


In [32]:
app_train = train.copy()
app_test = test.copy()

In [33]:
# Combining training and test data so that they get equal treatments.
app_train["set"]= "train"
app_test["set"]="test"
app_test["TARGET"] = np.nan

In [40]:
train_test= app_train.append(app_test, ignore_index =True)
train_test.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,set
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.014900,"Stone, brick",WEDNESDAY,0.972168,0.972168,0.972168,0.619141,0.624512,0.634277,train
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.071411,Block,MONDAY,0.984863,0.984863,0.984863,0.795898,0.798828,0.804199,train
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,train
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,train
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,train


## Entity Set

In [42]:
#Creating an empty Entity set 
es = ft.EntitySet(id ="credit_clients")

In [47]:
# dataframes containing a unique index
es = es.entity_from_dataframe(entity_id = "train_test",
                              dataframe = train_test,
                             index = "SK_ID_CURR")
es = es.entity_from_dataframe(entity_id = "bureau",
                              dataframe = bureau,
                             index = "SK_ID_BUREAU")
es = es.entity_from_dataframe(entity_id = "previous_application",
                              dataframe = previous_application,
                             index = "SK_ID_PREV")

In [51]:
# dataframes containing no unique index
es = es.entity_from_dataframe(entity_id = "credit_card_balance",
                              dataframe = credit_card_balance,
                              make_index = True,
                             index = "credit_card_balance_id")
es = es.entity_from_dataframe(entity_id = "installments_payments",
                              dataframe = installments_payments,
                              make_index = True,
                             index = "installments_payments_id")
es = es.entity_from_dataframe(entity_id = "POS_CASH_balance",
                              dataframe = POS_CASH_balance,
                              make_index = True,
                             index = "POS_CASH_balance_id")
es = es.entity_from_dataframe(entity_id = "bureau_balance",
                              dataframe = bureau_balance,
                              make_index = True,
                             index = "bureau_balance_id")

## Realationships

In [54]:
# Defining relationships among entities
traintest_bureau = ft.Relationship(es["train_test"]["SK_ID_CURR"],
                                  es["bureau"]["SK_ID_CURR"])

bureau_bureaubalance = ft.Relationship(es["bureau"]["SK_ID_BUREAU"],
                                  es["bureau"]["SK_ID_BUREAU"])


traintest_previousapplication = ft.Relationship(es["train_test"]["SK_ID_CURR"],
                                  es["previous_application"]["SK_ID_CURR"])


previousapplication_POSCASHbalance = ft.Relationship(es["previous_application"]["SK_ID_PREV"],
                                  es["POS_CASH_balance"]["SK_ID_PREV"])

previousapplication_installmentspayments = ft.Relationship(es["previous_application"]["SK_ID_PREV"],
                                  es["installments_payments"]["SK_ID_PREV"])
previousapplication_creditcardbalance = ft.Relationship(es["previous_application"]["SK_ID_PREV"],
                                  es["credit_card_balance"]["SK_ID_PREV"])

In [57]:
# Adding the defined relationships
es = es.add_relationships([traintest_bureau,bureau_bureaubalance,
                           bureau_bureaubalance,
                          traintest_previousapplication,
                          previousapplication_POSCASHbalance,
                           previousapplication_installmentspayments,
                           previousapplication_creditcardbalance
                          ])

2018-09-05 00:23:13,454 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: bureau.SK_ID_BUREAU -> bureau.SK_ID_BUREAU>


In [58]:
es

Entityset: credit_clients
  Entities:
    train_test [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 18]
    previous_application [Rows: 1001, Columns: 37]
    credit_card_balance [Rows: 1001, Columns: 24]
    installments_payments [Rows: 1001, Columns: 9]
    POS_CASH_balance [Rows: 1001, Columns: 9]
    bureau_balance [Rows: 1001, Columns: 4]
  Relationships:
    bureau.SK_ID_CURR -> train_test.SK_ID_CURR
    bureau.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous_application.SK_ID_CURR -> train_test.SK_ID_CURR
    POS_CASH_balance.SK_ID_PREV -> previous_application.SK_ID_PREV
    installments_payments.SK_ID_PREV -> previous_application.SK_ID_PREV
    credit_card_balance.SK_ID_PREV -> previous_application.SK_ID_PREV

## Feature Primitives

#### Aggregation

In [62]:
primitives = ft.list_primitives()
primitives[primitives["type"]=='aggregation'].head(15)

,name,type,description
0,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
1,min,aggregation,Finds the minimum non-null value of a numeric feature.
2,max,aggregation,Finds the maximum non-null value of a numeric feature.
3,skew,aggregation,Computes the skewness of a data set.
4,time_since_last,aggregation,Time since last related instance.
5,mode,aggregation,Finds the most common element in a categorical feature.
6,median,aggregation,Finds the median value of any feature with well-ordered values.
7,all,aggregation,Test if all values are 'True'.
8,mean,aggregation,Computes the average value of a numeric feature.
9,avg_time_between,aggregation,Computes the average time between consecutive events.


#### Transformation

In [60]:
primitives[primitives["type"]=='transform'].head(15)

,name,type,description
19,months,transform,Transform a Timedelta feature into the number ...
20,day,transform,Transform a Datetime feature into the day.
21,isin,transform,"For each value of the base feature, checks whe..."
22,mod,transform,Creates a transform feature that divides two f...
23,weeks,transform,Transform a Timedelta feature into the number ...
24,and,transform,"For two boolean values, determine if both valu..."
25,multiply,transform,Creates a transform feature that multplies two...
26,time_since,transform,Calculates time since the cutoff time.
27,minutes,transform,Transform a Timedelta feature into the number ...
28,absolute,transform,Absolute value of base feature.
